In [116]:
import pandas as pd
print(pd.__name__)

pandas


In [117]:
import numpy as np

In [118]:
import matplotlib.pyplot as plt
%matplotlib inline

In [119]:
plt.style.use('default')

In [120]:
import seaborn as sns
print(sns.__version__)

0.12.2


In [121]:
import plotly
import plotly.express as px
print(plotly.__version__)

5.13.0


In [122]:
diabetes = pd.read_csv('data/diabetes_data.csv')
diabetes.head()

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6       98             58             33      190  34.0   
1            2      112             75             32        0  35.7   
2            2      108             64              0        0  30.8   
3            8      107             80              0        0  24.6   
4            7      136             90              0        0  29.9   

   DiabetesPedigreeFunction  Age  Outcome  Gender  
0                     0.430   43        0  Female  
1                     0.148   21        0  Female  
2                     0.158   21        0  Female  
3                     0.856   34        0  Female  
4                     0.210   50        0  Female

Pregnancies — количество беременностей.

Glucose — концентрация глюкозы в плазме через два часа при пероральном тесте на толерантность к глюкозе.

BloodPressure — диастолическое артериальное давление (мм рт. ст.).

SkinThickness — толщина кожной складки трицепса (мм).

Insulin — двухчасовой сывороточный инсулин (ме Ед/мл).

BMI — индекс массы тела

DiabetesPedigreeFunction — функция родословной диабета (чем она выше, тем выше шанс наследственной заболеваемости).

Age — возраст.

Outcome — наличие диабета (0 — нет, 1 — да).

In [123]:
diabetes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 778 entries, 0 to 777
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               778 non-null    int64  
 1   Glucose                   778 non-null    int64  
 2   BloodPressure             778 non-null    int64  
 3   SkinThickness             778 non-null    int64  
 4   Insulin                   778 non-null    int64  
 5   BMI                       778 non-null    float64
 6   DiabetesPedigreeFunction  778 non-null    float64
 7   Age                       778 non-null    int64  
 8   Outcome                   778 non-null    int64  
 9   Gender                    778 non-null    object 
dtypes: float64(2), int64(7), object(1)
memory usage: 60.9+ KB


In [124]:
display(diabetes.tail())# последние строки

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
773            6      103             72             32      190  37.7   
774            1       71             48             18       76  20.4   
775            0      117              0              0        0  33.8   
776            4      154             72             29      126  31.3   
777            5      147             78              0        0  33.7   

     DiabetesPedigreeFunction  Age  Outcome  Gender  
773                     0.324   55        0  Female  
774                     0.323   22        0  Female  
775                     0.932   44        0  Female  
776                     0.338   37        0  Female  
777                     0.218   65        0  Female

In [125]:
dupl_columns = list(diabetes.columns)
dupl_columns.remove('Outcome')

mask = diabetes.duplicated(subset=dupl_columns)
diabetes_duplicates = diabetes[mask]
print(f'Число найденных дубликатов: {diabetes_duplicates.shape[0]}')

Число найденных дубликатов: 10


In [126]:
diabetes_dedupped = diabetes.drop_duplicates(subset=dupl_columns)
print(f'Результирующее число записей: {diabetes_dedupped.shape[0]}')

Результирующее число записей: 768


In [139]:
#список неинформативных признаков
low_information_cols = [] 

#цикл по всем столбцам
for col in diabetes_dedupped.columns:
    #наибольшая относительная частота в признаке
    top_freq = diabetes_dedupped[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = diabetes_dedupped[col].nunique() / diabetes_dedupped[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')

Gender: 100.0% одинаковых значений


In [140]:
information_diabetes= diabetes_dedupped.drop(low_information_cols, axis=1)
print(f'Результирующее число признаков: {information_diabetes.shape[1]}')

Результирующее число признаков: 9


In [141]:
def diabetes_nan(x):
    if x==0:
        return np.nan
    else:
        return x

In [142]:
information_diabetes['Insulin'] = information_diabetes['Insulin'].apply(diabetes_nan)
information_diabetes['Glucose'] = information_diabetes['Glucose'].apply(diabetes_nan)
information_diabetes['BloodPressure'] = information_diabetes['BloodPressure'].apply(diabetes_nan)
information_diabetes['SkinThickness'] = information_diabetes['SkinThickness'].apply(diabetes_nan)
information_diabetes['BMI'] = information_diabetes['BMI'].apply(diabetes_nan)

In [143]:
print(information_diabetes.Insulin.isnull().mean().round(2))

0.49


In [144]:
#8.4
#создаем копию исходной таблицы
drop_data = information_diabetes.copy()
#задаем минимальный порог: вычисляем 70% от числа строк
thresh = drop_data.shape[0]*0.7
#удаляем столбцы, в которых более 30% (100-70) пропусков
drop_data = drop_data.dropna(thresh=thresh, axis=1)
#отображаем результирующую долю пропусков
drop_data.isnull().mean()

Pregnancies                 0.000000
Glucose                     0.006510
BloodPressure               0.045573
SkinThickness               0.295573
BMI                         0.014323
DiabetesPedigreeFunction    0.000000
Age                         0.000000
Outcome                     0.000000
dtype: float64

In [145]:
# 8.5
m = drop_data.shape[1]
drop_data = drop_data.dropna(thresh=m-2, axis=0)
print(drop_data.shape[0])


761
